<a href="https://colab.research.google.com/github/jaimebaldeon/TSP-evolutionary-algorithm/blob/main/TSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
# Only needed to import the file into colab, needs only to be run once
from google.colab import files
uploaded = files.upload()


Saving tour29.csv to tour29.csv


In [ ]:
filename = next(iter(towns_csv))
matrix = genfromtxt(filename,delimiter=',')
matrix

In [65]:
import numpy as np
import statistics
import random

class TSP:
  def __init__(self, numVertices):
    #self.vertices = [n for n in range(1, numVertices+1)] #to accomplish natural values for vertices
    self.vertices = [n for n in range(numVertices)]
    matrix = np.random.randint(100, 500, size=(numVertices, numVertices))
    np.fill_diagonal(matrix, 0)
    self.distanceMatrix = matrix

class Individual:
  def __init__(self, tsp):
    self.order = np.random.permutation(tsp.vertices)
    self.α = 0.05

def fitness(tsp, ind):
  value = 0
  ii = 0
  for i in ind.order:
    # Sum up total distance for the individual's sequence of nodes 
    while (ii < ind.order.size - 1):
      pairVertices = ind.order[ii:ii+2]
      value += tsp.distanceMatrix[pairVertices[0], pairVertices[1]]
      ii += 1 
  # add the distance between first and last node
  value += tsp.distanceMatrix[ind.order[-1], ind.order[0]]
  return value

def optimize(tsp):
  λ = 100 # population size
  μ = 100 # offspring size
  iter =100

  # Initialization
  population = initialize(tsp,λ)
  offspring = []

  # Print initial Candidate solutions
  for ind in population:
    print("Candidate solution:  ", ind.order)

  # Print initial Population evaluation
  fitnesses = [fitness(tsp, ind) for ind in population]
  print("\nMean fitness:  ", statistics.mean(fitnesses))
  print("Best fitness:  ", min(fitnesses))

  for i in range(iter):
    # Recombination
    for ind in range(μ):
      # Selection
      parent1 = selection(tsp, population)
      parent2 = selection(tsp, population)
      offspring.append(recombination(parent1, parent2))
      mutate(offspring[ind])

    # Mutation
    for ind in population:
      mutate(ind)    

    # Elimination
    population = elimination(population, offspring, tsp)

    fitnesses = [fitness(tsp, ind) for ind in population]
    print(f'---------------------------------  ITERATION {i} ---------------------------------')
    print("Mean fitness:  ", statistics.mean(fitnesses))
    print("Best fitness:  ", min(fitnesses))

  # Print final Candidate solutions
  for ind in population:
    print("Candidate solution:  ", ind.order)
    print("Candidate mutation probability:  ", ind.α)


def initialize(tsp, λ):
  population = []
  for i in range(λ):
    population.append(Individual(tsp))
  return population

def mutate(individual):
  if np.random.rand() < individual.α:
    i = random.randrange(len(individual.order))
    j = random.randrange(len(individual.order))
    if i < j:
      individual.order[i:j] = individual.order[i:j][::-1]
    else:
      individual.order[j:i] = individual.order[j:i][::-1]

def recombination(parent1, parent2):
  child = parent1
  # Select crossover points randomly
  i = random.randrange(len(parent1.order))
  j = random.randrange(len(parent1.order))
  # Perform order crossover recombination to create the child's order
  if i < j:  
    child.order = order_crossover(parent1.order, parent2.order, i, j)
  else:  
    child.order = order_crossover(parent1.order, parent2.order, j, i)  
  # Set child parameters
  β = 2 * np.random.rand() - 0.5 # coefficient between (-0.5, 1.5)
  child.α = parent1.α + β * (parent2.α - parent1.α)
  return child

def order_crossover(parent1, parent2, i,j):
  # Create loop effect over array
  p1 = np.concatenate((parent1, parent1), axis=None)
  p2 = np.concatenate((parent2, parent2), axis=None)
  # Create empty child order
  child = np.full(len(parent2), None) 
  # Fill child with random selected sequence from parent1
  child[i:j] = p1[i:j]
  # Set pointers to the second crossover point in parent2 (jj) and child (ii)
  ii, jj = j, j
  # The recombination stops when all nodes in parent2 sequence have been added to the child
  while (jj < j + len(parent2)):
    if ii > len(parent2)-1:
      # when child's pointer reaches the end of the array go to position 0
      ii = 0
    if p2[jj] not in child: 
      # Missing nodes are added to the child
      child[ii] = p2[jj]
      ii += 1 # when added a new node, shift child's pointer to the left
    jj += 1
  return child

def selection(tsp, population):
  k = 5
  candidates = random.sample(population, k)
  fitnesses = [fitness(tsp, ind) for ind in candidates]
  selected = fitnesses.index(min(fitnesses))
  return candidates[selected]

def elimination(population, offspring, tsp):
  # λ+μ elimination
  combined = population + offspring
  ranked_individuals = sorted([(fitness(tsp, ind), ind) for ind in combined], key=lambda ind: ind[0])[0:len(population)]
  return [ind for fit, ind in ranked_individuals]

In [ ]:
tsp = TSP(5)
population=initialize(tsp,15)
for i in population:
  print("Candidate solution:  ", i.order)
  print("Objective value: ", fitness(tsp, i))

parent1 = selection(tsp, population)
parent2 = selection(tsp, population)
print(f'\n Selected parent {parent1.order} y fitness: {fitness(tsp, parent1)}')
print(f'\n Selected parent {parent2.order} y fitness: {fitness(tsp, parent2)}')

offspring = recombination(parent1, parent2)
print(f'\n Offspring recombination {offspring.order} y fitness: {fitness(tsp, offspring)}')

mutate(offspring)
print(f'\n Offspring mutated {offspring.order} y fitness: {fitness(tsp, offspring)}')

# Mutation
for ind in population:
  mutate(ind)    

off = [offspring]
# Elimination
population = elimination(population, off, tsp)
for i in population:
  print("Candidate solution:  ", i.order)
  print("Objective value: ", fitness(tsp, i))


In [106]:
tsp = TSP(5)

#print("Vertices:  ", tsp.vertices)
print(type(tsp.distanceMatrix))
print("Distance:  \n", tsp.distanceMatrix)

optimize(tsp)

"""best = elimination(pop, pop, tsp)
for i in best:
  print("Candidate solution:  ", i.order)
  print("Objective value: ", fitness(tsp, i))

fitnesses = [fitness(tsp, ind) for ind in best]
print("\nMean fitness:  ", statistics.mean(fitnesses))
print("Best fitness:  ", min(fitnesses))
"""

<class 'numpy.ndarray'>
Distance:  
 [[  0 333 355 142 147]
 [372   0 493 303 375]
 [291 476   0 250 180]
 [132 227 216   0 162]
 [416 124 298 164   0]]
Candidate solution:   [3 1 0 2 4]
Candidate solution:   [0 2 3 4 1]
Candidate solution:   [1 4 2 3 0]
Candidate solution:   [0 3 2 4 1]
Candidate solution:   [1 0 3 2 4]
Candidate solution:   [4 0 1 3 2]
Candidate solution:   [3 1 4 0 2]
Candidate solution:   [3 1 4 2 0]
Candidate solution:   [3 0 1 4 2]
Candidate solution:   [0 3 1 2 4]
Candidate solution:   [0 1 4 2 3]
Candidate solution:   [4 3 0 2 1]
Candidate solution:   [3 2 1 4 0]
Candidate solution:   [2 1 4 3 0]
Candidate solution:   [0 3 4 2 1]
Candidate solution:   [1 2 0 4 3]
Candidate solution:   [2 3 0 1 4]
Candidate solution:   [1 3 2 0 4]
Candidate solution:   [3 0 2 1 4]
Candidate solution:   [4 0 1 2 3]
Candidate solution:   [1 4 3 0 2]
Candidate solution:   [1 4 3 0 2]
Candidate solution:   [4 1 3 2 0]
Candidate solution:   [3 2 1 0 4]
Candidate solution:   [2 1 3 0 

KeyboardInterrupt: ignored